In [1]:
import geopandas as gpd
import pandas as pd
import folium
import re
from folium.plugins import MarkerCluster

In [2]:
def load_data():
    # DETER ALERTS (GEODF)
    alerts = gpd.read_file('../data/deter-amz-public-2024jul10/deter-amz-deter-public.shp', encoding='utf-8')
    alerts.loc[alerts['CLASSNAME'] == 'DEGRDACAO', 'CLASSNAME'] = 'DEGRADACAO'
    alerts = alerts[~(alerts['CLASSNAME'] == 'CORTE_SELETIVO')]
    alerts['VIEW_DATE'] = pd.to_datetime(alerts['VIEW_DATE'])
    alerts['ANO'] = alerts['VIEW_DATE'].dt.year
    alerts['MES'] = alerts['VIEW_DATE'].dt.month
    alerts['MES/ANO'] = alerts['VIEW_DATE'].dt.strftime('%Y-%m')

    # DETER ALERTS (CSV)
    df_deter = pd.DataFrame(alerts)
    df_deter = df_deter.drop(columns=['FID', 'QUADRANT', 'PATH_ROW', 'SENSOR', 'SATELLITE', 'geometry'])

    
    #IBGE DATA
    legal_amazon = gpd.read_file('../data/brazilian_legal_amazon/brazilian_legal_amazon.shp',encoding='utf-8')
    states = gpd.read_file('../data/states_legal_amazon/states_legal_amazon.shp',encoding='utf-8')
    
    ac = gpd.read_file('../data/malhas_regionais_ibge/AC_Municipios_2022/AC_Municipios_2022.shp', encoding='utf-8')
    am = gpd.read_file('../data/malhas_regionais_ibge/AM_Municipios_2022/AM_Municipios_2022.shp', encoding='utf-8')
    ap = gpd.read_file('../data/malhas_regionais_ibge/AP_Municipios_2022/AP_Municipios_2022.shp', encoding='utf-8')
    ma = gpd.read_file('../data/malhas_regionais_ibge/MA_Municipios_2022/MA_Municipios_2022.shp', encoding='utf-8')
    mt = gpd.read_file('../data/malhas_regionais_ibge/MT_Municipios_2022/MT_Municipios_2022.shp', encoding='utf-8')
    pa = gpd.read_file('../data/malhas_regionais_ibge/PA_Municipios_2022/PA_Municipios_2022.shp', encoding='utf-8')
    ro = gpd.read_file('../data/malhas_regionais_ibge/RO_Municipios_2022/RO_Municipios_2022.shp', encoding='utf-8')
    rr = gpd.read_file('../data/malhas_regionais_ibge/RR_Municipios_2022/RR_Municipios_2022.shp', encoding='utf-8')
    to = gpd.read_file('../data/malhas_regionais_ibge/TO_Municipios_2022/TO_Municipios_2022.shp', encoding='utf-8')

    df_cities = pd.concat([ac, am, ap, ma, mt, pa, ro, rr, to])
    df_cities.rename(columns={'CD_MUN':'GEOCODIBGE'}, inplace=True)

    c_units = gpd.read_file('../data/conservation_units_legal_amazon/conservation_units_legal_amazon.shp',encoding='utf-8')
    c_units.rename(columns={'nome':'UC'},inplace=True)
    
    return alerts, df_deter, legal_amazon, states, df_cities, c_units

alerts, df_deter, legal_amazon, states, df_cities, c_units = load_data()

In [3]:
# LOADING TEXTS (ENGLISH AND PORTUGUESE)

df_texts = pd.read_csv('../texts/texts_deter.csv', sep='§', engine='python')
english = {list(df_texts['Key'])[i]: list(df_texts['English'])[i] for i in range(len(list(df_texts['Key'])))}
portuguese = {list(df_texts['Key'])[i]: list(df_texts['Portuguese'])[i] for i in range(len(list(df_texts['Key'])))}

classes_deter_en = {'CICATRIZ_DE_QUEIMADA': 'Forest Fire Scar',
          'DESMATAMENTO_CR': 'Deforestation with Exposed Soil',
          'DESMATAMENTO_VEG': 'Deforestation with Vegetation',
          'MINERACAO': 'Mining',
          'DEGRADACAO': 'Degradation',
          'CS_DESORDENADO': 'Selective Logging Type 1 (Disordered)',
          'CS_GEOMETRICO': 'Selective Logging Type 2 (Geometric)',
}

classes_deter_pt = {'CICATRIZ_DE_QUEIMADA': 'Cicatriz de incêndio florestal',
          'DESMATAMENTO_CR': 'Desmatamento com solo exposto',
          'DESMATAMENTO_VEG': 'Desmatamento com Vegetação',
          'MINERACAO': 'Mineração',
          'DEGRADACAO': 'Degradação',
          'CS_DESORDENADO': 'Corte Seletivo Tipo 1 (Desordenado)',
          'CS_GEOMETRICO': 'Corte Seletivo Tipo 2 (Geométrico)',
}

estados = {
    "MT": "Mato Grosso",
    "PA": "Pará",
    "AM": "Amazonas",
    "RO": "Rondônia",
    "MA": "Maranhão",
    "RR": "Roraima",
    "AC": "Acre",
    "TO": "Tocantins",
    "AP": "Amapá"
}

def get_texts(lang):
    if lang == "English":
        return classes_deter_en, english
    else:
        return classes_deter_pt, portuguese

dict_classes, texts = get_texts("English")

In [4]:
def get_centroids(geo_df, mode=2, crs='EPSG:31982'):

    if mode==0:
        centroids = geo_df.copy()
        centroids["centroid"] = centroids.geometry.centroid
        centroids["latitude"] = centroids.centroid.y
        centroids["longitude"] = centroids.centroid.x
        return centroids

    if mode==1:
        geo_df_proj = geo_df.copy()
        geo_df_proj.to_crs(crs)
        
        geo_df_proj['centroid'] = geo_df_proj.geometry.centroid
        geo_df_proj['latitude'] = geo_df_proj.centroid.y
        geo_df_proj['longitude'] = geo_df_proj.centroid.x
        
        centroids = gpd.GeoDataFrame(geo_df_proj, geometry='centroid', crs=crs)
    
        centroids = centroids.to_crs(geo_df.crs)
        return centroids

    if mode==2:
        df = geo_df.copy()
        df['representative_point'] = df.geometry.representative_point()
        df['latitude'] = df['representative_point'].apply(lambda p: p.y)
        df['longitude'] = df['representative_point'].apply(lambda p: p.x)
        return df

def folium_map_init():
    map = folium.Map(location=[-7.25, -60], zoom_start=4)

    # folium.TileLayer(
    #     tiles='https://tiles.stadiamaps.com/tiles/stamen_terrain/{z}/{x}/{y}{r}.png', 
    #     attr='&copy; <a href="https://www.stadiamaps.com/" target="_blank">Stadia Maps</a> &copy; <a href="https://www.stamen.com/" target="_blank">Stamen Design</a> &copy; <a href="https://openmaptiles.org/" target="_blank">OpenMapTiles</a> &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',
    #     name='Stamen Terrain'
    # ).add_to(map)

    return map

def folium_add_markers(container, df_data, geo_df, get_centroid_mode, df_deter, key, popup_title_column, popup_total_area_text='Área Total Afetada:',total_area_column='AREAMUNKM'):

    all_classes = sorted(df_deter['CLASSNAME'].unique())
    df_centroids = get_centroids(geo_df,get_centroid_mode)
    
    for idx, row in df_data.iterrows():

        # Get Centroids for marker location
        coords = df_centroids.loc[df_centroids[key] == row[key]].iloc[0]

        # Variable of statistics
        info = ''
        
        # Get classes statistics from the deter dataframe
        if row['AREAMUNKM']>0:
            df_stats = df_deter[df_deter[key] == row[key]]
            df_stats_summed = df_stats.groupby('CLASSNAME')['AREAMUNKM'].sum().reset_index()
            df_stats_complete = pd.DataFrame({'CLASSNAME': all_classes})
            df_stats_complete = df_stats_complete.merge(df_stats_summed, on='CLASSNAME', how='left').fillna(0)
            df_stats_complete['DESC'] = df_stats_complete['CLASSNAME'].map(dict_classes)
            df_stats_complete = df_stats_complete.sort_values(by='AREAMUNKM', ascending=False)
    
            total = df_stats_complete['AREAMUNKM'].sum()
            
            # Calculates percentage of every class
            for ind, lin in df_stats_complete.iterrows():
                perc = (lin['AREAMUNKM'] * 100) / total
                info += f"{lin['DESC']}: {lin['AREAMUNKM']:.0f} km² ({perc:.2f}%)<br>"
        
        popup_text = f"""
        <div style='white-space: nowrap;'>
        <span style='font-size: 16px; font-weight: bold;'>{row[popup_title_column]}</span><br><br> {popup_total_area_text} {row[total_area_column]:.0f} km²<br><br> {info}
        </div>
        """

        # Add marker on Map or MarkerCluster (container)
        folium.Marker(
            location=[coords['latitude'], coords['longitude']],
            popup=popup_text,
            icon=folium.Icon(color='red', icon='triangle-exclamation', prefix='fa')
        ).add_to(container)
    
    return container

In [5]:
def save_map(file_name,map):
    map.save(f"../Visualizations/DETER/Maps/{file_name}.html")

### States Map:

In [6]:
def states_map():
    
    ############# Data Preparation #############
    # legal_amazon = gpd.read_file('data/brazilian_legal_amazon/brazilian_legal_amazon.shp',encoding='utf-8')
    # states = gpd.read_file('data/states_legal_amazon/states_legal_amazon.shp',encoding='utf-8')
    
    df_deter = alerts.copy()
    gb_uf = df_deter.groupby('UF')['AREAMUNKM'].sum().sort_values(ascending=False)
    gb_uf = pd.DataFrame(gb_uf)
    
    gb_uf['NOME_ESTADO'] = gb_uf.index.map(estados)
    gb_uf['NOME_SIGLA'] = gb_uf['NOME_ESTADO'] + ' (' + gb_uf.index + ')' 
    gb_uf = gb_uf.reset_index()
    states_copy = states.copy()
    states_copy = states_copy.rename(columns={'sigla':'UF'})



    #############       Folium       #############
    
    map = folium_map_init()
    
    style_states = {'fillOpacity':0 ,'color' : '#117306', 'weight': 2}
    folium.GeoJson(states_copy, name = 'States', style_function= lambda x: style_states).add_to(map)
    
    style_legal_amazon = {'fillOpacity':0 ,'color' : '#117306', 'weight': 3}
    folium.GeoJson(legal_amazon, name = 'Legal Amazon', style_function= lambda x: style_legal_amazon).add_to(map)
    
    folium.Choropleth(geo_data=states_copy,
                      data=gb_uf,
                      columns=['UF', 'AREAMUNKM'],
                      key_on = 'feature.properties.UF',
                      fill_color = 'YlOrRd',
                      nan_fill_color = 'white',
                      bins=10,
                      highlight = True,
                      legend_name='Affected Area in km²',
                      name='Most Affected States').add_to(map)

    map = folium_add_markers(map, gb_uf, states_copy, 1, df_deter, 'UF', 'NOME_SIGLA', texts['total_dmg'], 'AREAMUNKM')
    
    folium.LayerControl().add_to(map)

    return map

In [7]:
dict_classes, texts = get_texts("English")
map = states_map()
save_map('States_EN',map)

C:\Users\lucas\AppData\Local\Temp\ipykernel_1352\1314333218.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_df_proj['centroid'] = geo_df_proj.geometry.centroid
C:\Users\lucas\AppData\Local\Temp\ipykernel_1352\1314333218.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_df_proj['latitude'] = geo_df_proj.centroid.y
C:\Users\lucas\AppData\Local\Temp\ipykernel_1352\1314333218.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_df_proj['longitude'] = geo_df_proj.centroid.x


In [8]:
dict_classes, texts = get_texts("Portuguese")
map = states_map()
save_map('States_PT',map)

C:\Users\lucas\AppData\Local\Temp\ipykernel_1352\1314333218.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_df_proj['centroid'] = geo_df_proj.geometry.centroid
C:\Users\lucas\AppData\Local\Temp\ipykernel_1352\1314333218.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_df_proj['latitude'] = geo_df_proj.centroid.y
C:\Users\lucas\AppData\Local\Temp\ipykernel_1352\1314333218.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geo_df_proj['longitude'] = geo_df_proj.centroid.x


### Cities Map

In [9]:
def cities_map(filter=[]):
    
    ############# Data Preparation #############

    geocodibge = alerts.drop_duplicates(subset='MUNICIPALI').set_index('MUNICIPALI')['GEOCODIBGE']
    sum_areamunkm = alerts.groupby('MUNICIPALI')['AREAMUNKM'].sum().reset_index()
    sum_areamunkm['GEOCODIBGE'] = sum_areamunkm['MUNICIPALI'].map(geocodibge)

    merge = pd.merge(df_cities, sum_areamunkm, on='GEOCODIBGE', how='left')

    if len(filter)>0:
        merge = merge[merge['SIGLA_UF'].isin(filter)]
    
    # merge[merge['AREAMUNKM'].isna()]
    # 226 cidades não contém avisos. Esses valores ausentes serão preenchidos com 0.
    merge['AREAMUNKM'].fillna(0, inplace=True)

    #############       Folium       #############
    map = folium_map_init()

    style_cities = {'fillOpacity':0 ,'color' : '#117306', 'weight': 1}
    folium.GeoJson(merge, name = 'Cities', style_function= lambda x: style_cities).add_to(map)

    style_states = {'fillOpacity':0 ,'color' : '#117306', 'weight': 2}
    folium.GeoJson(states, name = 'States', style_function= lambda x: style_states).add_to(map)
    
    style_legal_amazon = {'fillOpacity':0 ,'color' : '#117306', 'weight': 3}
    folium.GeoJson(legal_amazon, name = 'Legal Amazon', style_function= lambda x: style_legal_amazon).add_to(map)
    
    folium.Choropleth(geo_data=merge.to_json(),
                  name='Choropleth',
                  data=merge,
                  columns=['GEOCODIBGE', 'AREAMUNKM'],
                  key_on = 'feature.properties.GEOCODIBGE',
                  fill_color = 'YlOrRd',
                  nan_fill_color = 'white',
                  highlight = True,
                  legend_name='Affected Area in km²').add_to(map)

    marker_cluster = MarkerCluster().add_to(map)
    marker_cluster = folium_add_markers(marker_cluster, merge, merge, 2, alerts, 'GEOCODIBGE', 'NM_MUN', texts['total_dmg'], 'AREAMUNKM')
    folium.LayerControl().add_to(map)

    return map

In [10]:
lst_states = list(df_cities['SIGLA_UF'].unique())
lst_states

['AC', 'AM', 'AP', 'MA', 'MT', 'PA', 'RO', 'RR', 'TO']

In [11]:
dict_classes, texts = get_texts("English")

for i in range(len(lst_states)):
    filter = []
    filter.append(lst_states[i])
    map = cities_map(filter)
    map_name = 'Cities_EN_' + filter[0]
    save_map(map_name,map)
    print(map_name + ' saved.')

Cities_EN_AC saved.
Cities_EN_AM saved.
Cities_EN_AP saved.
Cities_EN_MA saved.
Cities_EN_MT saved.
Cities_EN_PA saved.
Cities_EN_RO saved.
Cities_EN_RR saved.
Cities_EN_TO saved.


In [12]:
dict_classes, texts = get_texts("English")

map = cities_map([])
map_name = 'All_Cities_EN'
save_map(map_name,map)
print(map_name + ' saved.')

All_Cities_EN saved.


In [13]:
dict_classes, texts = get_texts("Portuguese")

for i in range(len(lst_states)):
    filter = []
    filter.append(lst_states[i])
    map = cities_map(filter)
    map_name = 'Cities_PT_' + filter[0]
    save_map(map_name,map)
    print(map_name + ' saved.')

Cities_PT_AC saved.
Cities_PT_AM saved.
Cities_PT_AP saved.
Cities_PT_MA saved.
Cities_PT_MT saved.
Cities_PT_PA saved.
Cities_PT_RO saved.
Cities_PT_RR saved.
Cities_PT_TO saved.


In [14]:
dict_classes, texts = get_texts("Portuguese")

map = cities_map([])
map_name = 'All_Cities_PT'
save_map(map_name,map)
print(map_name + ' saved.')

All_Cities_PT saved.


### C Units

In [15]:
def normalize_string(s):
    s = s.strip()
    s = re.sub(r'\s+', ' ', s)
    s = re.sub(r'[^\w\s]', '', s)
    s = s.upper()
    return s

def c_units_map():
    ############# Data Preparation #############
    c_units_copy = c_units.copy()
    c_units_copy['UC'] = c_units_copy['UC'].apply(normalize_string)
    
    alerts_uc = alerts[alerts['UC'].notna()].copy()
    dic_correcao = {'FLORESTA NACIONAL DE ALTAMIRA': 'FLORESTA NACIONAL ALTAMIRA', 
                    'FLORESTA NACIONAL DE CAXIUANÂ': 'FLORESTA NACIONAL DE CAXIUANÃ', 
                    'FLORESTA NACIONAL DO AMANA': 'FLORESTA NACIONAL DO AMANÁ',
                    'FLORESTA NACIONAL DO BOM FUTURO': 'FLORESTA NACIONAL DE BOM FUTURO',
                    'FLORESTA NACIONAL DO ITACAIUNAS': 'FLORESTA NACIONAL DE ITACAIUNAS',
                    'FLORESTA NACIONAL DO JATUARANA': 'FLORESTA NACIONAL DE JATUARANA',
                    'FLORESTA NACIONAL DO PURUS': 'RESERVA EXTRATIVISTA DO MÉDIO PURÚS',
                    'FLORESTA NACIONAL DO TAPAJÓS': 'FLORESTA NACIONAL DE TAPAJÓS',
                    'FLORESTA NACIONAL DO TAPIRAPÉAQUIRI': 'FLORESTA NACIONAL DE TAPIRAPÉAQUIRI',
                    'FLORESTA NACIONAL MAPIÁ  INAUINI': 'FLORESTA NACIONAL DE MAPIÁINAUINÍ',
                    'PARQUE NACIONAL SERRA DA CUTIA': 'PARQUE NACIONAL DA SERRA DA CUTIA',
                    'RESERVA BIOLÓGICA NASCENTES DA SERRA DO CACHIMBO': 'RESERVA BIOLÓGICA NASCENTES SERRA DO CACHIMBO',
                    'RESERVA EXTRATIVISTA DO ALTO JURUÁ': 'RESERVA EXTRATIVISTA ALTO JURUÁ',
                    'RESERVA EXTRATIVISTA DO ALTO TARAUACÁ': 'RESERVA EXTRATIVISTA ALTO TARAUACÁ',
                    'RESERVA EXTRATIVISTA DO BAIXO JURUÁ': 'RESERVA EXTRATIVISTA BAIXO JURUÁ',
                    'RESERVA EXTRATIVISTA DO CIRIACO': 'RESERVA EXTRATIVISTA DO CIRIÁCO',
                    'RESERVA EXTRATIVISTA DO LAGO DO CUNIÃ': 'RESERVA EXTRATIVISTA LAGO DO CUNIÃ',
                    'RESERVA EXTRATIVISTA DO MÉDIO JURUÁ': 'RESERVA EXTRATIVISTA MÉDIO JURUÁ',
                    'RESERVA EXTRATIVISTA DO RIO CAJARI': 'RESERVA EXTRATIVISTA RIO CAJARI',
                    'RESERVA EXTRATIVISTA DO RIO DO CAUTÁRIO': 'RESERVA EXTRATIVISTA RIO CAUTÁRIO',
                    'RESERVA EXTRATIVISTA DO RIO OURO PRETO': 'RESERVA EXTRATIVISTA RIO OURO PRETO',
                    'RESERVA EXTRATIVISTA RIO UNINI': 'RESERVA EXTRATIVISTA DO RIO UNINI',
                    'RESERVA EXTRATIVISTA TAPAJÓSARAPIUNS': 'RESERVA EXTRATIVISTA TAPAJÓS ARAPIUNS',
                    'RESERVA EXTRATIVISTA TAPAJÓS-ARAPIUNS': 'RESERVA EXTRATIVISTA TAPAJÓS ARAPIUNS',
                    'RESERVA EXTRATIVISTA TERRA GRANDE  PRACUÚBA': 'RESERVA EXTRATIVISTA TERRA GRANDE PRACUUBA',
                    'RESERVA EXTRATIVISTA TERRA GRANDE - PRACUÚBA': 'RESERVA EXTRATIVISTA TERRA GRANDE PRACUUBA',
                    'ÁREA DE PROTEÇÃO AMBIENTAL DOS MEANDROS DO RIO ARAGUAIA': 'ÁREA DE PROTEÇÃO AMBIENTAL MEANDROS DO ARAGUAIA',
                    'ÁREA DE RELEVANTE INTERESSE ECOLÓGICO SERINGAL NOVA ESPERANÇA': 'ÁREA DE RELEVANTE INTERESSE ECOLÓGICA SERINGAL NOVA ESPERANÇA',
                    'ESTAÇÃO ECOLÓGICA JUAMI-JAPURÁ': 'ESTAÇÃO ECOLÓGICA JUAMIJAPURÁ',
                    'FLORESTA NACIONAL DE BALATA-TUFARI': 'FLORESTA NACIONAL DE BALATATUFARI',
                    'FLORESTA NACIONAL DE SARACÁ-TAQUERA': 'FLORESTA NACIONAL DE SARACÁTAQUERA',
                    'FLORESTA NACIONAL MAPIÁ - INAUINI': 'FLORESTA NACIONAL DE MAPIÁINAUINÍ',
                    'RESERVA EXTRATIVISTA AUATÍ-PARANÁ': 'RESERVA EXTRATIVISTA AUATÍPARANÁ',
                    'RESERVA EXTRATIVISTA DO CAZUMBÁ-IRACEMA': 'RESERVA EXTRATIVISTA DO CAZUMBÁIRACEMA',
                    'RESERVA EXTRATIVISTA GURUPÁ-MELGAÇO': 'RESERVA EXTRATIVISTA GURUPÁMELGAÇO',
                    'RESERVA EXTRATIVISTA IPAÚ-ANILZINHO': 'RESERVA EXTRATIVISTA IPAÚANILZINHO'}
    alerts_uc['UC'] = alerts_uc['UC'].replace(dic_correcao)
    gc_uc = alerts_uc.groupby('UC')['AREAMUNKM'].sum().reset_index()
    gc_uc['UC'] = gc_uc['UC'].replace(dic_correcao)
    
    # Not found conservation units in c_units_copy dataframe
    # lst_dif=[]
    # for item in list(gc_uc['UC']):
    #     if item not in list(c_units_copy['UC']):
    #         lst_dif.append(item)
    # lst_dif # ['ESTAÇÃO ECOLÓGICA DE CARACARAÍ', 'ESTAÇÃO ECOLÓGICA DE IQUÊ']
    
    # Inserindo unidades de conservação que faltam no geodataframe "c_units_copy" nas coordenadas do primeiro alerta encontrado nos dados do DETER
    def uc_geodf(estacao):
        primeiro_alerta = alerts[alerts['UC'] == estacao]['geometry'].iloc[0]
        ponto_representativo = primeiro_alerta
        novo_registro = {
            'UC': estacao,
            'geometry': ponto_representativo
        }
        return gpd.GeoDataFrame([novo_registro], crs=c_units_copy.crs)
    
    c_units_copy = pd.concat([c_units_copy, uc_geodf('ESTAÇÃO ECOLÓGICA DE CARACARAÍ')], ignore_index=True)
    c_units_copy = pd.concat([c_units_copy, uc_geodf('ESTAÇÃO ECOLÓGICA DE IQUÊ')], ignore_index=True)
    
    merge_ucs = pd.merge(c_units_copy, gc_uc, on='UC', how='left').fillna(0)


    
    #############       Folium       #############

    map = folium_map_init()

    style_legal_amazon = {'fillOpacity':0 ,'color' : '#117306', 'weight': 3}
    folium.GeoJson(legal_amazon, name = 'Legal Amazon', style_function= lambda x: style_legal_amazon).add_to(map)

    style_states = {'fillOpacity':0 ,'color' : '#117306', 'weight': 2}
    folium.GeoJson(states, name = 'States', style_function= lambda x: style_states).add_to(map)

    style_ucs = {'fillOpacity':0 ,'color' : '#3d1601', 'weight': 1}
    folium.GeoJson(c_units_copy, name = 'Conservation Units', style_function= lambda x: style_ucs).add_to(map)


    folium.Choropleth(geo_data=merge_ucs.to_json(),
                  name='Choropleth',
                  data=merge_ucs,
                  columns=['UC', 'AREAMUNKM'],
                  key_on = 'feature.properties.UC',
                  fill_color = 'YlOrRd',
                  nan_fill_color = 'white',
                  highlight = True,
                  legend_name='Affected Area in km²').add_to(map)
    
    marker_cluster = MarkerCluster().add_to(map)

    marker_cluster = folium_add_markers(marker_cluster,merge_ucs,merge_ucs, 2, alerts_uc, 'UC', 'UC', texts['total_dmg'], 'AREAMUNKM')

    folium.LayerControl().add_to(map)
    return map

In [16]:
dict_classes, texts = get_texts("English")
map = c_units_map()
save_map('C_Units_EN',map)

In [17]:
dict_classes, texts = get_texts("Portuguese")
map = c_units_map()
save_map('C_Units_PT',map)